### Download the Russian Wikipedia dump, process it and get txt files for pre-training.

In [0]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### WikiExtractor library

In [0]:
!git clone https://github.com/attardi/wikiextractor.git

Cloning into 'wikiextractor'...
remote: Enumerating objects: 613, done.
remote: Total 613 (delta 0), reused 0 (delta 0), pack-reused 613
Receiving objects: 100% (613/613), 1.24 MiB | 15.50 MiB/s, done.
Resolving deltas: 100% (352/352), done.


In [0]:
cd wikiextractor

/content/wikiextractor


### Download ruWikipedia dump

In [0]:
!wget http://dumps.wikimedia.org/ruwiki/latest/ruwiki-latest-pages-articles.xml.bz2

In [0]:
!mkdir -p 'data/wiki'

### Start dump processing

In [0]:
!python3 WikiExtractor.py \
    -o data/wiki \
    --quiet \
    --no_templates \
    --processes 8 \
    ruwiki-latest-pages-articles.xml.bz2

In [0]:
# save intermediate calculations
!tar -zcf ruwiki.tar.gz data

In [0]:
files = 'AA  AD	AG  AJ	AM  AP	AS  AV	AY  BB	BE  BH	BK  BN	BQ  BT	BW  BZ	CC  CF AB  AE	AH  AK	AN  AQ	AT  AW	AZ  BC	BF  BI	BL  BO	BR  BU	BX  CA	CD  CG AC  AF	AI  AL	AO  AR	AU  AX	BA  BD	BG  BJ	BM  BP	BS  BV	BY  CB	CE  CH'
categories = files.split()

### Download post processing script

In [0]:
!wget https://gist.githubusercontent.com/snakers4/e0b0e68904db65671ca979639b337f7b/raw/dd5f7caabdb451047355a5536bc6df05000b00cb/process_wikipedia.py

--2020-05-22 11:44:31--  https://gist.githubusercontent.com/snakers4/e0b0e68904db65671ca979639b337f7b/raw/dd5f7caabdb451047355a5536bc6df05000b00cb/process_wikipedia.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3871 (3.8K) [text/plain]
Saving to: ‘process_wikipedia.py’

process_wikipedia.p 100%[===================>]   3.78K  --.-KB/s    in 0s      

2020-05-22 11:44:31 (67.1 MB/s) - ‘process_wikipedia.py’ saved [3871/3871]



### Get csv from the archive and break it into parts

In [0]:
path_to_archive = '../data/ruwiki.tar.gz'
path_save_archive = '../data/csv'

chunk = 5
amount = len(categories)
bar = tqdm( enumerate( range(0, amount, chunk), start=1), total=amount/chunk)
for i, k in bar:
  !mkdir -p 'data/wiki'
  unit = categories[k : k + chunk]
  for t, name in enumerate(unit, start=1):
    bar.set_description(f'Categories: #{t}; Name:{name}')
    dir_data = f'data/wiki/{name}'
    !tar -zxf {path_to_archive} {dir_data}
  !python3 process_wikipedia.py
  rename = f'data/ruwiki{i}.csv'
  !mv 'data/ruwiki_2018_09_25.csv' {rename}
  archive = f'data/ruwiki{i}.tar.gz'
  !tar -zcf {archive} {rename}
  !cp {archive} {path_save_archive}
  !rm -R 'data'

100% 500/500 [07:27<00:00,  1.12it/s]
100% 500/500 [09:14<00:00,  1.11s/it]
100% 500/500 [09:09<00:00,  1.10s/it]
100% 500/500 [08:58<00:00,  1.08s/it]
100% 500/500 [07:22<00:00,  1.13it/s]
100% 500/500 [08:51<00:00,  1.06s/it]
100% 500/500 [09:46<00:00,  1.17s/it]
100% 500/500 [09:04<00:00,  1.09s/it]
100% 500/500 [07:26<00:00,  1.12it/s]
100% 500/500 [09:50<00:00,  1.18s/it]
100% 500/500 [10:21<00:00,  1.24s/it]
100% 458/458 [08:36<00:00,  1.13s/it]



### Get txt format for pre-training

In [0]:
path_save_txt = '../data/txt'

bar = tqdm(range(1, 13))
for i in bar:
  data_dir = f'{path_save_archive}/ruwiki{i}.tar.gz'
  bar.set_description('operation #1: unzip')
  !tar -xf {data_dir}
  listsen = []
  bar.set_description('operation #2: preparing a list')
  for chunk in pd.read_csv(f'data/ruwiki{i}.csv', chunksize=50000):  
    chunk.dropna(how='any', inplace = True)
    chunk['len'] = list(map(lambda sen: len(sen), chunk['sentence']))
    values = chunk[chunk['len'] >= 4]['sentence'].values
    listsen.extend(values)
  bar.set_description('operation #3: list to txt')
  with open(f'data/listsen{i}.txt', 'w') as filehandle:
    for listitem in listsen:
      filehandle.write('%s\n' % listitem)
  bar.set_description('operation #4: archive and save')
  data_dir = f'data/listsen{i}.txt'
  archive = f'{path_save_txt}/ruwiki_txt_{i}.tar.gz'
  !tar -zcf {archive} {data_dir}